In [0]:
import networkx as nx
from networkx.algorithms import community
import community


In [0]:
g = nx.read_edgelist('amazon.ungraph.txt',create_using = nx.Graph())
nodes = g.nodes
edges = g.edges

##### 1. Which item(s) are the most popular?  
###### Calculate PageRank of all items:

In [0]:
page_ranks = nx.pagerank(g, alpha=0.85, max_iter=1000)

In [0]:
pageRank = {}
for j in nodes:
    pageRank[j] = page_ranks.get(j)

In [0]:
Top_5_PR = []
Top_5_PR = sorted(pageRank.items(), key=lambda k: k[1], reverse=True)[0:5]
Top_5_PR

[('548091', 0.00022486903624139567),
 ('458358', 0.0001067249264521988),
 ('222074', 9.535038238390979e-05),
 ('291117', 7.951003014915984e-05),
 ('45146', 7.371981749432723e-05)]

##### 1.2 Which item(s) are the most popular? 

In [0]:
top_nodes_Ave = ['548091','515301','296016','186356','436020']
top_product = nx.Graph()


In [0]:
neighbours = {}
for pr in top_nodes_Ave:
    neighbours[pr] = []
for pr in top_nodes_Ave:
    temp = []
    for u,v in g.edges():
        if u == pr:
            neighbours[pr].append(v)
            top_product.add_edge(u, v)
        if v == pr:
            top_product.add_edge(v, u)

In [0]:
for key, value in neighbours.items():
    print("The production ID", key, "has ", len(value), " neighbors")

The production ID 548091 has  549  neighbors
The production ID 515301 has  228  neighbors
The production ID 296016 has  212  neighbors
The production ID 186356 has  142  neighbors
The production ID 436020 has  197  neighbors


In [0]:
nx.write_gml(top_product, "top_product.gml")

##### 1.3 Which item(s) are purchased along with the most popular item(s)?Does the most popular items and their co-purchased items 
##### 1.4 belong to the same community?

In [0]:
#louvain community detection
community_louvain = community.best_partition(g)

In [0]:
inv_community = {}
for key1, value1 in community_louvain.items():
        inv_community.setdefault(value1, []).append(key1)

In [0]:

for key, value in neighbours.items():
    for key1, value1 in inv_community.items():
        if key in value1:
            print('The product ', key, ' is inside the community ', key1)
            set_neighbor = set(value)
            set_community = set(value1)
            matched_production = set_neighbor & set_community
            print('A total ', len(matched_production), " products from it's neighbor are also located in this community")
            matching_rate = float("{0:.2f}".format(len(matched_production)/len(set_neighbor) * 100))
            print('The matching rate is ', matching_rate, '%' )
            print('\n')
            
    

The product  548091  is inside the community  34
A total  470  products from it's neighbor are also located in this community
The matching rate is  85.61 %


The product  515301  is inside the community  56
A total  222  products from it's neighbor are also located in this community
The matching rate is  97.37 %


The product  296016  is inside the community  56
A total  207  products from it's neighbor are also located in this community
The matching rate is  97.64 %


The product  186356  is inside the community  56
A total  135  products from it's neighbor are also located in this community
The matching rate is  95.07 %


The product  436020  is inside the community  5
A total  196  products from it's neighbor are also located in this community
The matching rate is  99.49 %




##### Calculate the degree and eigenvector centrality

In [0]:
dict_degree_centrality = nx.degree_centrality(g)
dict_eigenvector_centrality = nx.eigenvector_centrality(g)

In [0]:
Top_all_PR = []
Top_all_PR = sorted(pageRank.items(), key=lambda k: k[1], reverse=True)
Top_all_DN = [] 
Top_all_DN = sorted(dict_degree_centrality.items(), key=lambda k: k[1], reverse=True)
Top_all_EV = [] 
Top_all_EV = sorted(dict_eigenvector_centrality.items(), key=lambda k: k[1], reverse=True)

In [0]:
Top_PR = {}
for i, v in enumerate(Top_all_PR):
    Top_PR[v[0]]=i
Top_DN = {}
for i_1, v_1 in enumerate(Top_all_DN):
    Top_DN[v_1[0]]=i_1
Top_EV = {}
for i_2, v_2 in enumerate(Top_all_EV):
    Top_EV[v_2[0]]=i_2

##### Calculate the average among all three methods, Page Rank, degree and eigenvector centrality
######  top average is the mean of postisions.

In [0]:
Top_Average = {}

for key_a, value_a in Top_PR.items():
    sum1 = Top_PR[key_a] + Top_DN[key_a] + Top_EV[key_a]
    Top_Average[key_a] = sum1/3
        
    

In [0]:
Top_1000_Ave = []
Top_1000_Ave = sorted(Top_Average.items(), key=lambda k: k[1])[0:1000]

In [0]:
Top_1000_Ave[0:10]

[('548091', 0.0),
 ('515301', 6.666666666666667),
 ('296016', 7.666666666666667),
 ('186356', 21.666666666666668),
 ('436020', 29.666666666666668),
 ('135898', 94.33333333333333),
 ('27832', 148.0),
 ('424153', 170.66666666666666),
 ('110214', 193.66666666666666),
 ('410716', 199.33333333333334)]

##### Calculate the most popular communities

In [0]:
Community_PR = {}
weight_num = {}
for key, value in inv_community.items():
    Community_PR[key] = 0
    weight_num[key] = 0
for index, value in enumerate(Top_1000_Ave):
    for key1, value1 in inv_community.items():
        if value[0] in value1:
            Community_PR[key1] +=1
            weight_num[key1] += (1000-(index+1))/1000


In [0]:
popular_communities = sorted(Community_PR.items(), key=lambda k: k[1], reverse=True)

In [0]:
weight_num[0]


6.677

In [0]:
top_communities = {}
top20_communities = []
for i in popular_communities:
    community_nodes = len(inv_community[i[0]])
    score = i[1]*weight_num[i[0]]/community_nodes
    #print(score)
    top_communities[i[0]]=score
top20_communities =  sorted(top_communities.items(), key=lambda k: k[1], reverse=True)[0:20]  

In [0]:
top20_communities

[(5, 3.7032492159227965),
 (34, 0.8600054683365677),
 (56, 0.6325738396624473),
 (19, 0.6021729640498898),
 (16, 0.3023230155210644),
 (131, 0.2725278106508876),
 (15, 0.20868599508599509),
 (7, 0.1823629112662014),
 (30, 0.16587146676852552),
 (61, 0.0892625732798519),
 (117, 0.0801930501930502),
 (81, 0.050661087866108796),
 (22, 0.048828692848769055),
 (55, 0.03314322497586095),
 (114, 0.026523838261919133),
 (52, 0.024263076923076925),
 (50, 0.020612174458738627),
 (68, 0.016435406698564595),
 (135, 0.015445350734094617),
 (27, 0.013959898154042007)]

In [0]:
Top_100_Ave = []
Top_100_Ave = sorted(Top_Average.items(), key=lambda k: k[1])[0:100]

In [0]:
top5_communities =  sorted(top_communities.items(), key=lambda k: k[1], reverse=True)[0:5]

In [0]:
top_100_located = {}
for m in top5_communities:
    top_100_located[m[0]] = 0

top_100_located  

{5: 0, 16: 0, 19: 0, 34: 0, 56: 0}

In [0]:
for i in Top_100_Ave:
    for j in top5_communities:
        if i[0] in inv_community[j[0]]:
            top_100_located[j[0]] +=1

In [0]:
top_100_located

{5: 59, 16: 4, 19: 8, 34: 5, 56: 9}

In [0]:
Top_10000_Ave = []
Top_10000_Ave = sorted(Top_Average.items(), key=lambda k: k[1])[0:10000]
top20_communities =  sorted(top_communities.items(), key=lambda k: k[1], reverse=True)[0:20]

##### build the functions for the recommendation system

In [0]:
def getNeighbours(item_id):
    neighbours = []
    for u,v in g.edges():
        if u == item_id:
            neighbours.append(v)
        if v == neighbours:
            neighbours.append(u)
    return neighbours

In [0]:
def recommendationSystem(item_id):
    recommented_items = []
    neighbours = getNeighbours(item_id)
    #print(neighbours)
    for v in neighbours:
        if v in Top_10000_Ave:
            for c in top20_communities:
                if v in inv_community[c[0]]:
                    recommented_items.append(v)
    return recommented_items
            
            
    
    
    

In [0]:
top5_communities

[(5, 3.7032492159227965),
 (34, 0.8600054683365677),
 (56, 0.6325738396624473),
 (19, 0.6021729640498898),
 (16, 0.3023230155210644)]